# Conversation with a ReAct Agent

In this example, we will show you
- How to create a new service function for the tool agent to use
- How to use the ServiceToolkit module pre-process the tool functions for LLMs
- How to use the built-in ReAct agent to solve a problem

## Prerequisites

- Follow [READMD.md](https://github.com/modelscope/agentscope) to install AgentScope. 
- Prepare a model configuration. AgentScope supports both local deployed model services (CPU or GPU) and third-party services. More details and example model configurations please refer to our [tutorial](https://modelscope.github.io/agentscope/en/tutorial/203-model.html).
- [Optional] A bing (or google) search API key is suggested to experience the web search function. Here we take bing search as an example. 

## Note

- The example is tested with the following models. For other models, you may need to adjust the prompt.
    - gpt-4
    - gpt-3.5-turbo
    - qwen-max

In [1]:
YOUR_MODEL_CONFIGURATION_NAME = "{YOUR_MODEL_CONFIGURATION_NAME}"

YOUR_MODEL_CONFIGURATION = {
    "model_type": "xxx", 
    "config_name": YOUR_MODEL_CONFIGURATION_NAME
    
    # ...
}

BING_API_KEY = "{BING_API_KEY}"

## Step 1: Customize a Service Function

Taking `execute_python_code` as an example, we will show how to create a new service function that can be processed by `ServiceToolkit` module.

In AgentScope, a service function should have the following structure:

- A well formatted docstring (Google style is recommended), which contains
    - A brief description of function in docstring.
    - A brief description of the input arguments.
    - Wrap the output and execution status into a `ServiceResponse` object.

The following is a simple example of a service function that executes Python code and captures the output. (Also, you can use the `execute_python_code` function in the `agentscope.service` module directly, which provide more features.)"

In [2]:
from agentscope.service import ServiceResponse, ServiceExecStatus
import sys, io

def execute_python_code(code: str) -> ServiceResponse:
    """
    Execute Python code and capture the output. Note you must `print` the output to get the result.
    Args:
        code (`str`):
            The Python code to be executed.
    """  # noqa

    # Create a StringIO object to capture the output
    old_stdout = sys.stdout
    new_stdout = io.StringIO()
    sys.stdout = new_stdout

    try:
        # Using `exec` to execute code
        exec(code)
    except Exception as e:
        # If an exception occurs, capture the exception information
        output = str(e)
        status = ServiceExecStatus.ERROR
    else:
        # If the execution is successful, capture the output
        output = new_stdout.getvalue()
        status = ServiceExecStatus.SUCCESS
    finally:
        # Recover the standard output
        sys.stdout = old_stdout

    # Wrap the output and status into a ServiceResponse object
    return ServiceResponse(status, output)

After defining the service function, we try to use `ServiceToolkit` to pre-process the tool functions for LLMs. Here, we take `execute_python_code` function as example and observe the JSON schema generated by `service_toolkit`.

In [3]:
from agentscope.service import ServiceToolkit

service_toolkit = ServiceToolkit()
service_toolkit.add(execute_python_code)

import json
print(json.dumps(service_toolkit.json_schemas, indent=4))

{
    "execute_python_code": {
        "type": "function",
        "function": {
            "name": "execute_python_code",
            "description": "Execute Python code and capture the output. Note you must `print` the output to get the result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The Python code to be executed."
                    }
                },
                "required": [
                    "code"
                ]
            }
        }
    }
}


## Step 2: Prepare all tool functions for the agent
Similar as above, we can create different service functions, or use the built-in functions in `agentscope.service` module as follows. More service functions can be found under the `agentscope.service` module.

In [4]:
from agentscope.service import (
    bing_search, # or google_search,
    read_text_file,
    write_text_file, 
)

# Deal with arguments that need to be input by developers
service_toolkit.add(bing_search, api_key=BING_API_KEY, num_results=3)
service_toolkit.add(read_text_file)
service_toolkit.add(write_text_file)

## Step 3: Create a React Agent

A ReAct Agent is built in AgentScope to show how to construct complex reasoning processes. It solves a problem by a loop of "thought", "action" and "observation" steps. 

- thought: analyze and decide next action (the tools to use)
- action: execute the action (the tool function)
- observation: observe the execution results

In AgentScope, we implement the above steps in a `ReActAgent` class. Let's first taking a look at its system prompt. 

In [5]:
from agentscope.agents import ReActAgent
import agentscope

agentscope.init(
    model_configs=YOUR_MODEL_CONFIGURATION,
    project="Conversation with ReActAgent",
)

agent = ReActAgent(
    name="assistant",
    model_config_name=YOUR_MODEL_CONFIGURATION_NAME,
    service_toolkit=service_toolkit, 
    verbose=True, # set verbose to True to show the reasoning process
)

print("#"*80)
print(agent.sys_prompt)
print("#"*80)

2024-04-16 09:52:31.694 | INFO     | agentscope.models:read_model_configs:181 - Load configs for model wrapper: qwen, gpt-4, ollama
2024-04-16 09:52:31.711 | INFO     | agentscope.utils.monitor:_create_monitor_table:341 - Init [monitor_metrics] as the monitor table
2024-04-16 09:52:31.712 | INFO     | agentscope.utils.monitor:_create_monitor_table:342 - Init [monitor_metrics_quota_exceeded] as the monitor trigger
2024-04-16 09:52:31.712 | INFO     | agentscope.utils.monitor:__init__:311 - SqliteMonitor initialization completed at [./runs/run_20240416-095137_jgmih8/agentscope.db]
2024-04-16 09:52:31.716 | INFO     | agentscope.models.model:__init__:200 - Initialize model by configuration [qwen]
2024-04-16 09:52:31.718 | INFO     | agentscope.utils.monitor:register:362 - Register metric [qwen-max.call_counter] to SqliteMonitor with unit [times] and quota [None]
2024-04-16 09:52:31.719 | INFO     | agentscope.utils.monitor:register:362 - Register metric [qwen-max.prompt_tokens] to SqliteM

As shown above, the system prompt is consisted of 

- the description of the agent
- the description of tool functions
- some notices for the large language models

Besides, when ReAct Agent generates "thought", we use the following prompt to guide the generation. It's wrapped into a message with `role` field as `"system"` to distinguish from the user's input.

```python
# print(agent.parser.format_instruction)
"""
You should respond a json object in a json fenced code block as follows:
```json
{
	"thought": "what you thought", 
	"speak": "what you speak", 
	"function": [
		{
			"name": "{function name}", 
			"arguments": {
				"{argument1 name}": xxx, 
				"{argument2 name}": xxx
			}
		}
	]
}```
"""
```

When the `function` field in the response dictionary is empty, the agent will end the "reasoning-acting" loop and return the final response to the user. 

More implementation details please refer to the [source code](../../src/agentscope/agents/react_agent.py).

## Step 4: Try to Solve a Problem with the ReAct Agent

Now, let's try to ask the example question in the ReAct Paper: `"Aside from the Apple Remote, what other device can control the program Apple Remote was originally designed to interact with?"` as follows

In [8]:
from agentscope.message import Msg

msg_question = Msg(
    name="user", 
    content="Aside from the Apple Remote, what other device can control the program Apple Remote was originally designed to interact with?", 
    role="user"
)

res = agent(msg_question)

The above example is very interesting! Because the ReAct algorithm is famous and its example question occurs in many different websites, the results in the first searching are all about ReAct algorithm, rather than the answer to the question. 

Knowing this, in the reasoning step of the second iteration, the ReAct agent decides to rewrite the query and re-search in the website. It finally finds the answer in the third iteration. 

The behavior of the ReAct agent is unexpected and interesting. It shows that the ReAct agent can learn from the previous results and adjust its behavior in the next iteration. 

You can use the following code to build a conversation with the ReAct agent, and we also provide [completed code](./code/conversation_with_react_agent.py). 

```python
from agentscope.agents import UserAgent

user = UserAgent(name="User")

x = None
while True:
    x = user(x)
    if x.content == "exit":
        break
    x = agent(x)
```

In the end, we print the memory of the agent to show the reasoning process.

In [8]:
print(json.dumps(agent.memory.get_memory(), indent=4, ensure_ascii=False))

[
    {
        "id": "d71ec3345ca3452697d33da946d3725f",
        "timestamp": "2024-03-26 19:19:57",
        "name": "system",
        "content": "You are a helpful assistant.\n\nThe following tool functions are available in the format of\n```\n{index}. {function name}: {function description}\n    {argument name} ({argument type}): {argument description}\n    ...\n```\n\nTool Functions:\n1. bing_search: Search question in Bing Search API and return the searching results\n\tquestion (string): The search query string.\n2. execute_python_code: Execute Python code and capture the output. Note you must `print` the output to get the result.\n\tcode (string): The Python code to be executed.\n3. read_text_file: Read the content of the text file.\n\tfile_path (string): The path to the text file to be read.\n4. write_text_file: Write content to a text file.\n\toverwrite (boolean): Whether to overwrite the file if it already exists.\n\tfile_path (string): The path to the file where content will 